In [1]:
#INTEL CORPORATION CONFIDENTIAL AND PROPRIETARY
#
#Copyright © 2021 Intel Corporation.
#
#This software and the related documents are Intel copyrighted
#materials, and your use of them is governed by the express
#license under which they were provided to you (License). Unless
#the License provides otherwise, you may not use, modify, copy,
#publish, distribute, disclose or transmit  this software or the
#related documents without Intel's prior written permission.
#
#This software and the related documents are provided as is, with
#no express or implied warranties, other than those that are
#expressly stated in the License.

In [2]:
# Import modules
import os
import sys

import time
import numpy as np
import matplotlib.pyplot as plt
import nxsdk.api.n2a as nx
# import nxsdk_modules.slayer.src as nxSlayer
import slayer_rec.src as nxSlayer
import ioProcess.ioProcess as io
from nxsdk.graph.processes.phase_enums import Phase

os.environ['SLURM'] = '1'
os.environ['PARTITION'] = 'nahuku32'

accComp = True
# accComp = False

threshold = 1 # 1, 2, 5, 10

In [3]:
class CUB_braille():

    def __init__(self, thr, train=True):
        self.data, self.labels = self.load_files(thr,train)
    

    def load_files(self,thr,train):
        if train:
            data = np.load('data/CUB_new_braille_data_train_th' + str(thr) + '.npy', allow_pickle=True) # (samples, timeStamp, addr)
            labels = np.load('data/CUB_new_braille_labels_train_th' + str(thr) + '.npy', allow_pickle=True)
        else:
            data = np.load('data/CUB_new_braille_data_test_th' + str(thr) + '.npy', allow_pickle=True)
            labels = np.load('data/CUB_new_braille_labels_test_th' + str(thr) + '.npy', allow_pickle=True)   

        return data, labels
    
    
    def __getitem__(self, index):
        addr      = self.data[index][:,1]
        timeStamp = self.data[index][:,0].astype(float) # convert to ms
        label     = self.labels[index]

        # print("addr.shape = ", addr.shape)
        # print("addr = ", addr)
        # print("timeStamp.shape = ", timeStamp.shape)
        # print("timestamp = ", timeStamp)
        # print("label = ", label)
        # print("Address min-max = ", addr.min(), addr.max())
        # print("Timestep min-max = ", timeStamp.min(), timeStamp.max())

        event = np.zeros((len(addr), 4))
        event[:, 0] = addr # integers
        event[:, 3] = timeStamp # float in ms
        
        return event, label
    
    
    def __len__(self):
        return len(self.data)
        # return 1

# Create nxSlayer Dataset instance

In [4]:
dataset = nxSlayer.auto.s2lDataset(dataset = CUB_braille(thr = threshold, train = False))

# Create network

In [5]:
net = nxSlayer.auto.Network('netsLoihi/spyNet_rec_th' + str(threshold) + '.net', probeOutput = accComp, noDelay = True)

# Compile, setup input-output, and run the network

In [6]:
# customizing allocation for layers where default compartment allocation is failing
customCompartmentPerCore = [
    [0, 10], # customize compartment allocation for input layer (start by putting half of the current estimate)
    [1, 0], # there is a hidden flatten layer that flattens out the input layer to put all neurons in the channel dim
    [2, 10], # customize compartment allocation for layer 1 (start by putting half of the current estimate)
    [3, 14], # customize compartment allocation for layer 2
]

# net.create()
net.create(customCompPerCore=customCompartmentPerCore)

Creating Network
|   Type   |  W  |  H  |  C  | ker | str | pad |delay|neurons/Core|cores|
|input     |   48|    1|    1|     |     |     |False|          10|    5|
|recurrent |    1|    1|  450|     |     |     |False|          10|   45|
|dense     |    1|    1|   28|     |     |     |False|          14|    2|
|Total                                                            |   53|



52.99999999999994

In [7]:
if not accComp:
    (iProbes, vProbes, sProbes) = net.outputLayer.probe([nx.ProbeParameter.COMPARTMENT_CURRENT, nx.ProbeParameter.COMPARTMENT_VOLTAGE, nx.ProbeParameter.SPIKE])

net.compile()

net.setupIO(dataset, numSnips=2, blankTime=100)

# snip_dir = ("/homes/lyes_khacef/Loihi_Projects/Tactile_Sensing_on_Loihi/slayer/src/snips")
# snip_process = net.board.createSnip(
#         phase = Phase.EMBEDDED_MGMT,
#         name = 'snip_reset',
#         includeDir = snip_dir,
#         cFilePath = snip_dir + '/myreset2.c', # myspiking2.c
#         funcName = 'layer_reset_mgmt_fx', # layer_reset_mgmt_fx (iomgmt.c -> myspiking2.c): without the if
#         guardName = 'do_reset', # do_reset (myspiking2.c)
#         lmtId = 2,
#         chipId = net.board.nxChips[0].id,
#     )

spikes = np.array(net.spikeData, dtype=object)

results = net.run()

Starting compilation ...
Completed compilation in 13.73 seconds
Using per sample spike time: 270steps (+ 100steps gap)
INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 10.212.98.104:43105
INFO:DRV:      Host server up..............Done 0.20s
INFO:DRV:      Encoding axons/synapses.....Done 0.58s
INFO:DRV:      Compiling Embedded snips....Done 0.62s
INFO:DRV:      Compiling MPDS Registers....Done 1.68ms
INFO:HST:  Args chip=0 cpu=0 /homes/lyes_khacef/python3_venv/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1649661511.4637787/launcher_chip0_lmt0.bin --chips=1 --epoch=0 --remote-relay=0 
INFO:HST:  Args chip=0 cpu=1 /homes/lyes_khacef/python3_venv/lib/python3.8/site-packages/nxsdk/driver/compilers/../../../temp/1649661511.4637787/launcher_chip0_lmt1.bin --chips=1 --epoch=0 --remote-relay=0 
INFO:HST:  Nx...
INFO:DRV:      Booting up..................Done 0.81s
INFO:DRV:      Encoding probes.............Done 0.99ms
INFO:DRV:      Transferring probes.

In [8]:
if not accComp:
    fig = plt.figure(3, figsize=(25,15), dpi=500)

    # Plot compartmentCurrents
    ax0 = plt.subplot(3,1,1)
    ih = iProbes.plot()

    # Plot compartmentVoltages
    ax1 = plt.subplot(3,1,2)
    vh = vProbes.plot()

    # Plot spike trains
    ax2 = plt.subplot(3,1,3)
    sh = sProbes.plot()

    ax2.set_xlim(ax0.get_xlim())

    plt.show()

In [9]:
if accComp:
    print(results.mean())

5.028042328042328


# Gather output population results

In [10]:
if accComp:
    results_sum = results.reshape(results.shape[0], 28, -1).sum(axis=2)

# Check accuracy

In [11]:
if accComp:
    labels = dataset.labels
    accuracy = nxSlayer.s2l.checkAccuracy(labels, results_sum)
    print('Final accuracy is {:.2f}%'.format(accuracy*100))

Final accuracy is 59.35%


In [12]:
dataset.labels

[22,
 19,
 20,
 18,
 8,
 17,
 19,
 9,
 7,
 25,
 9,
 10,
 22,
 19,
 6,
 3,
 8,
 0,
 7,
 0,
 23,
 13,
 15,
 3,
 1,
 9,
 25,
 8,
 13,
 7,
 23,
 20,
 21,
 6,
 24,
 12,
 20,
 0,
 2,
 21,
 13,
 6,
 21,
 17,
 5,
 18,
 21,
 6,
 17,
 19,
 13,
 15,
 10,
 12,
 2,
 12,
 17,
 24,
 8,
 6,
 7,
 0,
 12,
 20,
 10,
 0,
 19,
 12,
 10,
 23,
 24,
 26,
 18,
 17,
 11,
 14,
 22,
 22,
 15,
 8,
 10,
 16,
 16,
 11,
 13,
 0,
 20,
 15,
 24,
 23,
 5,
 24,
 21,
 24,
 24,
 18,
 16,
 0,
 17,
 18,
 26,
 5,
 26,
 5,
 3,
 12,
 12,
 23,
 25,
 15,
 13,
 14,
 5,
 18,
 5,
 21,
 23,
 18,
 3,
 16,
 23,
 25,
 7,
 17,
 6,
 22,
 6,
 7,
 6,
 7,
 12,
 25,
 20,
 1,
 0,
 16,
 25,
 10,
 0,
 16,
 1,
 24,
 24,
 7,
 24,
 23,
 11,
 8,
 25,
 16,
 14,
 16,
 4,
 10,
 7,
 13,
 7,
 10,
 4,
 22,
 8,
 14,
 1,
 12,
 5,
 15,
 22,
 21,
 14,
 25,
 10,
 13,
 4,
 24,
 9,
 6,
 14,
 26,
 16,
 11,
 15,
 12,
 11,
 4,
 7,
 5,
 21,
 13,
 21,
 20,
 23,
 21,
 26,
 22,
 16,
 24,
 2,
 10,
 8,
 21,
 19,
 9,
 3,
 25,
 20,
 2,
 12,
 22,
 16,
 26,
 9,
 22,
 16,
 13

# Debug

In [13]:
# # net.create()
# net.create(numLayers=4)
# net.compile()
# net.setupIO(dataset, numSnips=1)
# spikes = np.array(net.spikeData, dtype=object)
# results = net.run()

# # prints
# print(spikes.shape)
# print(np.unique(spikes))
# print(spikes)
# print(results)